<a href="https://colab.research.google.com/github/batarg1nroman/Batargin-Roman/blob/main/sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import sqlite3
import os
from datetime import datetime
import json
from typing import Dict, Any, List, Optional

def print_query_result(cursor, query, message=None):
    """Универсальная функция для выполнения и печати результатов запроса"""
    if message:
        print(f"\n{message}:")
    cursor.execute(query)
    result = cursor.fetchall()
    for row in result:
        print(row)
    return result


class DatabaseManager:
    """Менеджер базы данных для работы с проектами."""

    def __init__(self, db_name: str = 'projects.db'):
        self.conn = sqlite3.connect(db_name)
        self.create_tables()

    def create_tables(self):
        """Создает все необходимые таблицы в базе данных."""
        cursor = self.conn.cursor()

        cursor.execute("""
        CREATE TABLE IF NOT EXISTS teams (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            members TEXT NOT NULL
        )
        """)

        cursor.execute("""
        CREATE TABLE IF NOT EXISTS projects (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            type TEXT NOT NULL,
            name TEXT NOT NULL,
            start_date TEXT NOT NULL,
            end_date TEXT NOT NULL,
            budget REAL NOT NULL,
            status TEXT NOT NULL,
            team_id INTEGER NOT NULL,
            programming_languages TEXT,
            target_audience INTEGER,
            research_field TEXT,
            FOREIGN KEY (team_id) REFERENCES teams(id)
        )
        """)

        self.conn.commit()

    def save_team(self, name: str, members: List[str]) -> int:
        """Сохраняет команду в базу данных и возвращает её ID."""
        cursor = self.conn.cursor()
        members_json = json.dumps(members)

        cursor.execute(
            "INSERT INTO teams (name, members) VALUES (?, ?)",
            (name, members_json)
        )
        self.conn.commit()
        return cursor.lastrowid

    def save_project(self, project: 'Project') -> int:
        """Сохраняет проект в базу данных и возвращает его ID."""
        cursor = self.conn.cursor()

        team_id = self.save_team(
            project.team.name,
            project.team.members
        )

        # Базовые данные проекта
        project_data = {
            'type': project.__class__.__name__,
            'name': project.name,
            'start_date': project.startDate.isoformat(),
            'end_date': project.endDate.isoformat(),
            'budget': project.budget,
            'status': project.status,
            'team_id': team_id,
            # Устанавливаем None для всех специфических полей по умолчанию
            'programming_languages': None,
            'target_audience': None,
            'research_field': None
        }

        # Заполняем специфические поля в зависимости от типа проекта
        if isinstance(project, SoftwareProject):
            project_data['programming_languages'] = json.dumps(project.programming_languages)
        elif isinstance(project, MarketingProject):
            project_data['target_audience'] = project.target_audience
        elif isinstance(project, ResearchProject):
             project_data['research_field'] = project.research_field

        cursor.execute("""
        INSERT INTO projects (
            type, name, start_date, end_date, budget, status, team_id,
            programming_languages, target_audience, research_field
        ) VALUES (
            :type, :name, :start_date, :end_date, :budget, :status, :team_id,
            :programming_languages, :target_audience, :research_field
        )
        """, project_data)

        self.conn.commit()
        return cursor.lastrowid

    def load_project(self, project_id: int) -> Optional['Project']:
        """Загружает проект из базы данных по ID."""
        cursor = self.conn.cursor()

        cursor.execute("""
        SELECT projects.*, teams.name as team_name, teams.members
        FROM projects
        LEFT JOIN teams ON projects.team_id = teams.id
        WHERE projects.id = ?
        """, (project_id,))

        project_row = cursor.fetchone()
        if not project_row:
            return None

        columns = [desc[0] for desc in cursor.description]
        project_data = dict(zip(columns, project_row))

        data = {
            'name': project_data['name'],
            'start_date': project_data['start_date'],
            'end_date': project_data['end_date'],
            'budget': project_data['budget'],
            'status': project_data['status'],
            'team': {
            'name': project_data['team_name'],
            'members': json.loads(project_data['members'])
            }
        }

        if project_data['type'] == 'SoftwareProject':
            data['programming_languages'] = json.loads(project_data['programming_languages'])
        elif project_data['type'] == 'MarketingProject':
            data['target_audience'] = project_data['target_audience']
        elif project_data['type'] == 'ResearchProject':
            data['research_field'] = project_data['research_field']

        project_class = globals()[project_data['type']]
        return project_class.from_dict(data)

    def get_all_projects(self) -> List['Project']:
        """Возвращает список всех проектов из базы данных."""
        cursor = self.conn.cursor()
        cursor.execute("SELECT id FROM projects")
        project_ids = [row[0] for row in cursor.fetchall()]
        return [self.load_project(pid) for pid in project_ids]

    def close(self):
        """Закрывает соединение с базой данных."""
        self.conn.close()

class Team:
    """Класс команды проекта."""

    def __init__(self, name: str, members: List[str]):
        self.name = name
        self.members = members

    def to_dict(self) -> Dict[str, Any]:
        return {
            'name': self.name,
            'members': self.members
        }

    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> 'Team':
        return cls(
            name=data['name'],
            members=data['members']
        )

class Project:
    """Базовый класс проекта."""

    def __init__(self, name: str, startDate: datetime, endDate: datetime,
                 budget: float, status: str, team: Team):
        self.name = name
        self.startDate = startDate
        self.endDate = endDate
        self.budget = budget
        self.status = status
        self.team = team

    def to_dict(self) -> Dict[str, Any]:
        return {
            'type': self.__class__.__name__,
            'name': self.name,
            'start_date': self.startDate.isoformat(),
            'end_date': self.endDate.isoformat(),
            'budget': self.budget,
            'status': self.status,
            'team': self.team.to_dict()
        }

    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> 'Project':
        team = Team.from_dict(data['team'])
        return cls(
            name=data['name'],
            startDate=datetime.fromisoformat(data['start_date']),
            endDate=datetime.fromisoformat(data['end_date']),
            budget=data['budget'],
            status=data['status'],
            team=team
        )

class SoftwareProject(Project):
    """Класс программного проекта."""

    def __init__(self, name: str, startDate: datetime, endDate: datetime,
                 budget: float, status: str, team: Team, programming_languages: List[str]):
        super().__init__(name, startDate, endDate, budget, status, team)
        self.programming_languages = programming_languages

    def to_dict(self) -> Dict[str, Any]:
        data = super().to_dict()
        data.update({
            'programming_languages': self.programming_languages
        })
        return data

    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> 'SoftwareProject':
        team = Team.from_dict(data['team'])
        return cls(
            name=data['name'],
            startDate=datetime.fromisoformat(data['start_date']),
            endDate=datetime.fromisoformat(data['end_date']),
            budget=data['budget'],
            status=data['status'],
            team=team,
            programming_languages=data['programming_languages']
        )

class MarketingProject(Project):
    """Класс маркетингового проекта."""

    def __init__(self, name: str, startDate: datetime, endDate: datetime,
                 budget: float, status: str, team: Team, target_audience: int):
        super().__init__(name, startDate, endDate, budget, status, team)
        self.target_audience = target_audience

    def to_dict(self) -> Dict[str, Any]:
        data = super().to_dict()
        data.update({
            'target_audience': self.target_audience
        })
        return data

    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> 'MarketingProject':
        team = Team.from_dict(data['team'])
        return cls(
            name=data['name'],
            startDate=datetime.fromisoformat(data['start_date']),
            endDate=datetime.fromisoformat(data['end_date']),
            budget=data['budget'],
            status=data['status'],
            team=team,
            target_audience=data['target_audience']
        )

class ResearchProject(Project):
    """Класс исследовательского проекта."""

    def __init__(self, name: str, startDate: datetime, endDate: datetime,
                 budget: float, status: str, team: Team, research_field: str):
        super().__init__(name, startDate, endDate, budget, status, team)
        self.research_field = research_field

    def to_dict(self) -> Dict[str, Any]:
        data = super().to_dict()
        data.update({
            'research_field': self.research_field
        })
        return data

    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> 'ResearchProject':
        team = Team.from_dict(data['team'])
        return cls(
            name=data['name'],
            startDate=datetime.fromisoformat(data['start_date']),
            endDate=datetime.fromisoformat(data['end_date']),
            budget=data['budget'],
            status=data['status'],
            team=team,
            research_field=data['research_field']
        )

if __name__ == "__main__":
  # Удаляем старую базу данных, если существует
    if os.path.exists('library.db'):
        os.remove('library.db')

    # Создаем новую базу данных
    conn = sqlite3.connect('library.db')
    cursor = conn.cursor()

    try:
        # 1. Создание таблицы books
        cursor.execute("""
        CREATE TABLE books (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            title TEXT NOT NULL,
            author TEXT NOT NULL,
            year_published INTEGER,
            genre TEXT
        )
        """)

        # 2. Вставка данных
        books_data = [
            ('The Great Gatsby', 'F. Scott Fitzgerald', 1925, 'Fiction'),
            ('1984', 'George Orwell', 1949, 'Dystopian'),
            ('To Kill a Mockingbird', 'Harper Lee', 1960, 'Classic')
        ]
        cursor.executemany("""
        INSERT INTO books (title, author, year_published, genre)
        VALUES (?, ?, ?, ?)
        """, books_data)
        conn.commit()

        # 3. Выборка всех данных
        print_query_result(cursor, "SELECT * FROM books", "Все книги")

        # 4. Фильтрация данных (после 1950 года)
        print_query_result(cursor,
                         "SELECT * FROM books WHERE year_published > 1950",
                         "Книги после 1950 года")

        # 5. Использование оператора LIKE
        print_query_result(cursor,
                         "SELECT * FROM books WHERE title LIKE 'T%'",
                         "Книги, названия которых начинаются на 'T'")

        # 6. Сортировка данных по году
        print_query_result(cursor,
                         "SELECT * FROM books ORDER BY year_published ASC",
                         "Книги, отсортированные по году издания")

        # 7. Ограничение количества строк
        print_query_result(cursor,
                         "SELECT * FROM books ORDER BY title LIMIT 2",
                         "Первые 2 книги по названию")

        # 8. Агрегатные функции
        cursor.execute("SELECT COUNT(*) AS total_books FROM books")
        print(f"\nВсего книг: {cursor.fetchone()[0]}")

        # 9. Группировка данных
        print_query_result(cursor,
                         "SELECT genre, COUNT(*) AS book_count FROM books GROUP BY genre",
                         "Количество книг по жанрам")

        # 10. Обновление данных
        cursor.execute("UPDATE books SET year_published = 1948 WHERE title = '1984'")
        conn.commit()
        print("\nГод публикации книги '1984' обновлен на 1948")

        # 11. Удаление данных
        cursor.execute("DELETE FROM books WHERE title = 'The Great Gatsby'")
        conn.commit()
        print("Книга 'The Great Gatsby' удалена")

        # 12. Создание индекса
        cursor.execute("CREATE INDEX idx_author ON books(author)")
        conn.commit()
        print("\nИндекс для столбца author создан")

        # 13. Проверка существования индекса
        cursor.execute("SELECT name FROM sqlite_master WHERE type = 'index' AND name = 'idx_author'")
        print(f"Индекс существует: {bool(cursor.fetchone())}")

        # 14. Создание таблицы logs и триггера
        cursor.execute("""
        CREATE TABLE logs (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            book_id INTEGER,
            action TEXT,
            timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
        )
        """)

        cursor.execute("""
        CREATE TRIGGER log_new_book
        AFTER INSERT ON books
        BEGIN
            INSERT INTO logs (book_id, action) VALUES (NEW.id, 'INSERT');
        END
        """)
        conn.commit()
        print("\nТаблица logs и триггер log_new_book созданы")

        # 15. Транзакция (выполняется отдельно после предыдущих коммитов)
        try:
            conn.execute("BEGIN")

            cursor.execute("""
                INSERT INTO books (title, author, year_published, genre)
                VALUES ('Pride and Prejudice', 'Jane Austen', 1813, 'Classic')
            """)

            cursor.execute("""
                UPDATE books SET year_published = 1961
                WHERE title = 'To Kill a Mockingbird'
            """)

            conn.commit()
            print("\nТранзакция успешно выполнена")

        except Exception as e:
            print(f"Ошибка в транзакции: {e}")
            conn.rollback()

        # 16. Пользовательская функция
        def calculate_book_age(year_published):
            return datetime.now().year - year_published

        conn.create_function("book_age", 1, calculate_book_age)
        print_query_result(cursor,
                         "SELECT title, book_age(year_published) AS age FROM books",
                         "Возраст книг")

        # 17. Создание представления
        cursor.execute("""
        CREATE VIEW modern_books AS
        SELECT * FROM books WHERE year_published > 1950
        """)
        print("\nПредставление modern_books создано")

        # 18. Удаление таблицы books (пропускаем, чтобы сохранить данные для демонстрации)
        # cursor.execute("DROP TABLE books")
        # print("\nТаблица books удалена")

        # 19. Создание композитного индекса
        cursor.execute("CREATE INDEX idx_author_year ON books(author, year_published)")
        conn.commit()
        print("Композитный индекс создан")

    except sqlite3.Error as e:
        print(f"Ошибка SQLite: {e}")
    finally:
        conn.close()

    # 20. Удаление базы данных (закомментировано для сохранения результатов)
    # if os.path.exists('library.db'):
    #     os.remove('library.db')
    #     print("\nФайл базы данных library.db удален")
    # else:
    #     print("\nФайл базы данных library.db не существует")


    if os.path.exists('projects.db'):
        os.remove('projects.db')

    db = DatabaseManager()

    team = Team("Dev Team", ["Alice", "Bob", "Charlie"])

    software_project = SoftwareProject(
        name="Web App",
        startDate=datetime(2023, 1, 1),
        endDate=datetime(2023, 12, 31),
        budget=50000,
        status="in progress",
        team=team,
        programming_languages=["Python", "JavaScript"]
    )

    marketing_project = MarketingProject(
        name="Ad Campaign",
        startDate=datetime(2023, 2, 1),
        endDate=datetime(2023, 6, 30),
        budget=20000,
        status="planned",
        team=team,
        target_audience=100000
    )

    research_project = ResearchProject(
        name="AI Research",
        startDate=datetime(2023, 3, 1),
        endDate=datetime(2024, 3, 1),
        budget=75000,
        status="active",
        team=team,
        research_field="Artificial Intelligence"
    )

    db.save_project(software_project)
    db.save_project(marketing_project)
    db.save_project(research_project)

    print("\nЗагруженные проекты:")
    for project in db.get_all_projects():
        print(f"{project.__class__.__name__}: {project.name} (Бюджет: ${project.budget:,})")

    db.close()


Все книги:
(1, 'The Great Gatsby', 'F. Scott Fitzgerald', 1925, 'Fiction')
(2, '1984', 'George Orwell', 1949, 'Dystopian')
(3, 'To Kill a Mockingbird', 'Harper Lee', 1960, 'Classic')

Книги после 1950 года:
(3, 'To Kill a Mockingbird', 'Harper Lee', 1960, 'Classic')

Книги, названия которых начинаются на 'T':
(1, 'The Great Gatsby', 'F. Scott Fitzgerald', 1925, 'Fiction')
(3, 'To Kill a Mockingbird', 'Harper Lee', 1960, 'Classic')

Книги, отсортированные по году издания:
(1, 'The Great Gatsby', 'F. Scott Fitzgerald', 1925, 'Fiction')
(2, '1984', 'George Orwell', 1949, 'Dystopian')
(3, 'To Kill a Mockingbird', 'Harper Lee', 1960, 'Classic')

Первые 2 книги по названию:
(2, '1984', 'George Orwell', 1949, 'Dystopian')
(1, 'The Great Gatsby', 'F. Scott Fitzgerald', 1925, 'Fiction')

Всего книг: 3

Количество книг по жанрам:
('Classic', 1)
('Dystopian', 1)
('Fiction', 1)

Год публикации книги '1984' обновлен на 1948
Книга 'The Great Gatsby' удалена

Индекс для столбца author создан
Индекс 